In [97]:
from math import sqrt
from dataclasses import dataclass
import numpy as np
from scipy import stats

@dataclass
class CIResult:
    ci_low: float
    ci_high: float
    estimate: float
    se: float
    crit: float
    decision: str
    note: str = ""

@dataclass
class TestResult:
    stat: float
    crit: float | tuple
    p: float
    reject: bool
    df: float | None = None
    note: str = ""


In [98]:
# CI helper functions

def ci_two_sided_mean_t(xbar, s, n, alpha=0.05):
    """Two-sided CI for mean, sigma unknown (t)."""
    se = s / sqrt(n)
    df = n - 1
    tcrit = stats.t.ppf(1 - alpha/2, df)
    me = tcrit * se
    return CIResult(
        ci_low=xbar - me, ci_high=xbar + me, estimate=xbar, se=se, crit=tcrit,
        decision="Reject H0 if μ0 outside CI; else Fail to reject.",
        note=f"df={df}, ME={me:.4f}"
    )

def one_sided_bound_mean_t(xbar, s, n, alpha=0.05, side="upper"):
    """
    One-sided bound for mean (t).
    side='upper' => upper bound = xbar + t_(1-alpha,df)*SE   [left-tail test]
    side='lower' => lower bound = xbar - t_(1-alpha,df)*SE   [right-tail test]
    """
    se = s / sqrt(n)
    df = n - 1
    tcrit = stats.t.ppf(1 - alpha, df)
    if side == "upper":
        bound = xbar + tcrit * se
        return CIResult(float("-inf"), bound, xbar, se, tcrit, "Compare μ0 to upper bound", f"df={df}")
    else:
        bound = xbar - tcrit * se
        return CIResult(bound, float("inf"), xbar, se, tcrit, "Compare μ0 to lower bound", f"df={df}")

def ci_two_sided_prop(phat, n, alpha=0.05):
    """Two-sided CI for proportion (Wald)."""
    se = sqrt(phat * (1 - phat) / n)
    zcrit = stats.norm.ppf(1 - alpha/2)
    me = zcrit * se
    return CIResult(
        ci_low=phat - me, ci_high=phat + me, estimate=phat, se=se, crit=zcrit,
        decision="CI for true p", note=f"ME={me:.5f}"
    )


In [99]:
# Critical/p-value approach helpers

def t_test_one_mean(xbar, mu0, s, n, alpha=0.05, tails="two-sided"):
    se = s / sqrt(n)
    df = n - 1
    t = (xbar - mu0) / se
    if tails == "two-sided":
        p = 2 * (1 - stats.t.cdf(abs(t), df))
        crit = stats.t.ppf(1 - alpha/2, df)
        reject = abs(t) > crit
    elif tails == "right":
        p = 1 - stats.t.cdf(t, df)
        crit = stats.t.ppf(1 - alpha, df)
        reject = t > crit
    else:
        p = stats.t.cdf(t, df)
        crit = stats.t.ppf(alpha, df)
        reject = t < crit
    return TestResult(stat=t, crit=crit, p=p, reject=reject, df=df)

def z_test_one_prop(x, n, p0, alpha=0.05, tails="two-sided"):
    phat = x / n
    se0 = sqrt(p0 * (1 - p0) / n)
    z = (phat - p0) / se0
    if tails == "two-sided":
        p = 2 * (1 - stats.norm.cdf(abs(z)))
        crit = stats.norm.ppf(1 - alpha/2)
        reject = abs(z) > crit
    elif tails == "right":
        p = 1 - stats.norm.cdf(z)
        crit = stats.norm.ppf(1 - alpha)
        reject = z > crit
    else:
        p = stats.norm.cdf(z)
        crit = stats.norm.ppf(alpha)
        reject = z < crit
    return TestResult(stat=z, crit=crit, p=p, reject=reject, df=None)


In [100]:
# ANOVA (summary) & Chi-square GOF

def anova_from_summary(groups):
    """
    groups: list of (n_i, mean_i, s_i)
    """
    N = sum(n for n, _, _ in groups)
    k = len(groups)
    grand_mean = sum(n * m for n, m, _ in groups) / N
    SSB = sum(n * (m - grand_mean) ** 2 for n, m, _ in groups)
    SSW = sum((n - 1) * (s ** 2) for n, _, s in groups)
    dfb, dfw = k - 1, N - k
    MSB, MSW = SSB / dfb, SSW / dfw
    F = MSB / MSW
    p = stats.f.sf(F, dfb, dfw)
    return {"F": F, "p": p, "df_between": dfb, "df_within": dfw, "MSB": MSB, "MSW": MSW,
            "grand_mean": grand_mean, "SSB": SSB, "SSW": SSW}

def chisq_gof(observed, expected_probs):
    N = sum(observed)
    expected = np.array(expected_probs) * N
    chi2 = ((np.array(observed) - expected) ** 2 / expected).sum()
    df = len(observed) - 1
    p = stats.chi2.sf(chi2, df)
    return {"chi2": chi2, "df": df, "p": p, "expected": expected}


In [101]:
# CI(1) Email Campaign
n, xbar, s, mu0, alpha = 40, 52, 12, 50, 0.05
ci = ci_two_sided_mean_t(xbar, s, n, alpha)
print(ci)
print("Decision vs μ0=50:", "Fail to reject H0" if (ci.ci_low <= mu0 <= ci.ci_high) else "Reject H0")
print("Conclusion: Fail to reject H0. The new layout has no significant effect on average spend.")



CIResult(ci_low=np.float64(48.1622138140854), ci_high=np.float64(55.8377861859146), estimate=52, se=1.8973665961010275, crit=np.float64(2.0226909200367604), decision='Reject H0 if μ0 outside CI; else Fail to reject.', note='df=39, ME=3.8378')
Decision vs μ0=50: Fail to reject H0
Conclusion: Fail to reject H0. The new layout has no significant effect on average spend.


In [102]:
# Block 6: CI(2) Delivery Time (one-sided upper bound)
n, xbar, s, mu0, alpha = 25, 3.8, 0.9, 4.0, 0.05
upper = one_sided_bound_mean_t(xbar, s, n, alpha, side="upper")
print(upper)
print("Decision (left-tail):", "Fail to reject H0" if mu0 <= upper.ci_high else "Reject H0")
print("Conclusion: Fail to reject H0. Insufficient evidence that mean delivery time is below 4 days.")



CIResult(ci_low=-inf, ci_high=np.float64(4.107958774383697), estimate=3.8, se=0.18, crit=np.float64(1.7108820799094275), decision='Compare μ0 to upper bound', note='df=24')
Decision (left-tail): Fail to reject H0
Conclusion: Fail to reject H0. Insufficient evidence that mean delivery time is below 4 days.


In [103]:
# Block 7: CI(3) App feature adoption
n, x = 500, 290
phat = x / n
ci_p = ci_two_sided_prop(phat, n, alpha=0.05)
print(ci_p)
print("Decision vs p0=0.60:", "Fail to reject H0" if (ci_p.ci_low <= 0.60 <= ci_p.ci_high) else "Reject H0")
print("Conclusion: Fail to reject H0. Adoption rate is not significantly different from 60%.")



CIResult(ci_low=np.float64(0.5367384843372058), ci_high=np.float64(0.6232615156627941), estimate=0.58, se=0.02207260745811423, crit=np.float64(1.959963984540054), decision='CI for true p', note='ME=0.04326')
Decision vs p0=0.60: Fail to reject H0
Conclusion: Fail to reject H0. Adoption rate is not significantly different from 60%.


In [104]:
# Block 8: CI(4) Customer Satisfaction (lower bound for right-tail)
n, xbar, s, mu0, alpha = 100, 4.20, 0.80, 4.00, 0.01
df = n - 1
tcrit = stats.t.ppf(1 - alpha, df)
se = s / sqrt(n)
lower_bound = xbar - tcrit * se
print({"lower_bound": round(lower_bound, 3), "se": round(se, 3), "tcrit": round(tcrit, 3), "df": df})
print("Decision:", "Reject H0" if mu0 < lower_bound else "Fail to reject H0")
print("Conclusion: Reject H0. Mean satisfaction rating is significantly greater than 4.0.")



{'lower_bound': np.float64(4.011), 'se': 0.08, 'tcrit': np.float64(2.365), 'df': 99}
Decision: Reject H0
Conclusion: Reject H0. Mean satisfaction rating is significantly greater than 4.0.


In [105]:
# Block 9: CI(5) NFL Linebackers
n, xbar, s, mu0, alpha = 6, 239.3, 10.9, 230.0, 0.05
ci_lb = ci_two_sided_mean_t(xbar, s, n, alpha)
print(ci_lb)
print("Decision:", "Fail to reject H0" if (ci_lb.ci_low <= mu0 <= ci_lb.ci_high) else "Reject H0")
print("Conclusion: Fail to reject H0. No evidence that the mean weight differs from 230 lb.")



CIResult(ci_low=np.float64(227.86115152513378), ci_high=np.float64(250.73884847486624), estimate=239.3, se=4.449906366056108, crit=np.float64(2.570581835636314), decision='Reject H0 if μ0 outside CI; else Fail to reject.', note='df=5, ME=11.4388')
Decision: Fail to reject H0
Conclusion: Fail to reject H0. No evidence that the mean weight differs from 230 lb.


In [106]:
# Block 10: A) Retail Price (two-tailed t)
n, xbar, s, mu0, alpha = 12, 79.8, 9.6, 75.0, 0.05
res = t_test_one_mean(xbar, mu0, s, n, alpha, tails="two-sided")
print(res)
print("Conclusion:", "Reject H0" if res.reject else "Fail to reject H0, Mean basket value is not significantly different from $75.")




TestResult(stat=1.7320508075688763, crit=np.float64(2.200985160082949), p=np.float64(0.11117295276905725), reject=np.False_, df=11, note='')
Conclusion: Fail to reject H0, Mean basket value is not significantly different from $75.


In [107]:
# Block 11: B) Call-center Wait (right-tailed t)
n, xbar, s, mu0, alpha = 16, 2.8, 0.9, 2.5, 0.10
res = t_test_one_mean(xbar, mu0, s, n, alpha, tails="right")
print(res)
print("Conclusion:", "Reject H0" if res.reject else "Fail to reject H0 , No strong evidence that average wait exceeds 2.5 minutes.")


TestResult(stat=1.3333333333333326, crit=np.float64(1.3406056078504547), p=np.float64(0.10115929173681915), reject=np.False_, df=15, note='')
Conclusion: Fail to reject H0 , No strong evidence that average wait exceeds 2.5 minutes.


In [108]:
# Block 12: C) Email CTR (two-tailed z-test for p)
n, x, p0, alpha = 900, 78, 0.08, 0.05
res = z_test_one_prop(x, n, p0, alpha, tails="two-sided")
print(res)
print("Conclusion:", "Reject H0" if res.reject else "Fail to reject H0, Click-through rate is not significantly different from 8%")


TestResult(stat=0.7372097807744858, crit=np.float64(1.959963984540054), p=np.float64(0.4609947864255779), reject=np.False_, df=None, note='')
Conclusion: Fail to reject H0, Click-through rate is not significantly different from 8%


In [109]:
# Block 13: D) Delivery Reliability (two-tailed z-test at 1%)
n, x, p0, alpha = 1200, 1100, 0.95, 0.01
res = z_test_one_prop(x, n, p0, alpha, tails="two-sided")
print(res)
print("Conclusion:", "Reject H0 , On-time delivery rate is significantly lower than 95%." if res.reject else "Fail to reject H0")


TestResult(stat=-5.298129428260172, crit=np.float64(2.5758293035489004), p=np.float64(1.1699502810991191e-07), reject=np.True_, df=None, note='')
Conclusion: Reject H0 , On-time delivery rate is significantly lower than 95%.


In [110]:
# Block 14: Regional Sales ANOVA
north = np.array([82.50, 85.30, 79.80, 83.10, 81.40])
south = np.array([76.80, 74.20, 78.50, 75.90, 77.30])
west  = np.array([88.10, 90.50, 85.70, 92.30, 87.60])

groups = [north, south, west]
F, p = stats.f_oneway(*groups)
print("F =", F, "p =", p)
print("Decision @0.01:", "Reject H0" if p < 0.01 else "Fail to reject H0 ")



F = 42.27747989276148 p = 3.6850208754849267e-06
Decision @0.01: Reject H0


In [111]:
# Block 15: Regional Sales summaries + ANOVA
north = np.array([82.50, 85.30, 79.80, 83.10, 81.40])
south = np.array([76.80, 74.20, 78.50, 75.90, 77.30])
west  = np.array([88.10, 90.50, 85.70, 92.30, 87.60])

ns = (15, 12, 18)  # stated sample sizes
means = [north.mean(), south.mean(), west.mean()]
stds  = [north.std(ddof=1), south.std(ddof=1), west.std(ddof=1)]

print("Summaries:")
for region, n_i, m, s_i in zip(["North", "South", "West"], ns, means, stds):
    print(f"{region}: n={n_i}, mean={m:.2f}, std={s_i:.2f}")

groups_regions = [(ns[i], means[i], stds[i]) for i in range(3)]
a2 = anova_from_summary(groups_regions)
print("\nANOVA:", a2)
print("Decision @0.01:", "Reject H0 (≥1 region differs)" if a2["p"] < 0.01 else "Fail to reject H0")


Summaries:
North: n=15, mean=82.42, std=2.04
South: n=12, mean=76.54, std=1.61
West: n=18, mean=88.84, std=2.58

ANOVA: {'F': np.float64(116.61121682974006), 'p': np.float64(7.159367431778904e-18), 'df_between': 2, 'df_within': 42, 'MSB': np.float64(555.8940000000009), 'MSW': np.float64(4.767071428571423), 'grand_mean': np.float64(83.41999999999999), 'SSB': np.float64(1111.7880000000018), 'SSW': np.float64(200.21699999999979)}
Decision @0.01: Reject H0 (≥1 region differs)


In [112]:
# Block 16: Channel Mix Chi-square
observed = np.array([275, 185, 40])
expected = np.array([300, 150, 50])

chi2 = ((observed - expected)**2 / expected).sum()
df = len(observed) - 1
p = stats.chi2.sf(chi2, df)

print("Chi-square =", chi2, "df =", df, "p =", p)
print("Decision @0.05:", "Reject H0" if p < 0.05 else "Fail to reject H0, Sales channel mix has changed significantly from the expected 60–30–10%")



Chi-square = 12.25 df = 2 p = 0.002187491118182885
Decision @0.05: Reject H0
